In [1]:
import pandas as pd
import numpy as np

In [2]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])

In [3]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five

1. Using the daily stock data from 2010, return an array of the unique trading days in 2010.

In [21]:
df2010.loc[:, 'retdate'].unique()

array(['2010-04-07T00:00:00.000000000', '2010-04-13T00:00:00.000000000',
       '2010-06-11T00:00:00.000000000', '2010-11-23T00:00:00.000000000',
       '2010-02-01T00:00:00.000000000', '2010-11-22T00:00:00.000000000',
       '2010-08-23T00:00:00.000000000', '2010-06-10T00:00:00.000000000',
       '2010-06-04T00:00:00.000000000', '2010-04-12T00:00:00.000000000',
       '2010-04-06T00:00:00.000000000', '2010-08-09T00:00:00.000000000',
       '2010-02-02T00:00:00.000000000', '2010-11-08T00:00:00.000000000',
       '2010-02-16T00:00:00.000000000', '2010-02-17T00:00:00.000000000',
       '2010-11-09T00:00:00.000000000', '2010-02-03T00:00:00.000000000',
       '2010-08-20T00:00:00.000000000', '2010-06-07T00:00:00.000000000',
       '2010-04-05T00:00:00.000000000', '2010-04-15T00:00:00.000000000',
       '2010-04-01T00:00:00.000000000', '2010-04-29T00:00:00.000000000',
       '2010-06-17T00:00:00.000000000', '2010-06-03T00:00:00.000000000',
       '2010-08-18T00:00:00.000000000', '2010-08-30

2. Return the symbol, return date and the dollar volume traded for the highest dollar volume traded stocks in 2010 on the NYSE.

In [16]:
(df2010.loc[(df2010.loc[:, 'exch'] == 'NYSE'), :].
assign(dol_vol = df2010.loc[:, 'cls']*df2010.loc[:, 'vol']).
nlargest(1, 'dol_vol').loc[:, ['symb', 'retdate', 'dol_vol']])

,symb,retdate,dol_vol
553212,BRK.B,2010-02-12,2.415266e+10


3. Using the 2010 data, return the stock (symbol only) with the largest volume on Jan 11th that also appears on Dec 1st.

In [27]:
(df2010.loc[
    (df2010.loc[:, 'symb'].isin(df2010.loc[(df2010.loc[:, 'retdate'] == '2010-12-01'), 'symb'].unique()) &
    (df2010.loc[:, 'retdate'] == '2010-01-11')), :].
    nlargest(1, 'vol').
    loc[:, ['symb']])

,symb
468965,F


4. Using the 2010 data, return the stock symbol and a column called “HFlag” which is equal to 1 if the high - low is greater than 1 and zero otherwise. Only return those companies whose stock symbol begins or ends with “A”.

In [32]:
temp = df2010.copy().loc[(df2010.loc[:, 'symb'].str.upper().str.startswith('A')), :]
temp.loc[((temp.loc[:, 'high'] - temp.loc[:, 'low']) > 1), 'HFlag'] = 1
temp.loc[((temp.loc[:, 'high'] - temp.loc[:, 'low']) < 1), 'HFlag'] = 0
temp.loc[:, ['symb', 'HFlag']]

,symb,HFlag
0,AAME,0.0
1,AAON,0.0
2,AAPL,0.0
3,AAWW,0.0
4,AAXJ,0.0
...,...,...
814436,AYI,0.0
814437,AYR,0.0
814438,AZN,0.0
814439,AZO,1.0


5. Write a query which returns (a) the date, (b) closing price and (c) a flag (“gt30”) which is equal to “1” when the closing price is greater than $30.00 and “0” otherwise for “AAPL” in 2010.

In [35]:
temp = df2010.copy().loc[(df2010.loc[:, 'symb'] == 'AAPL'), :]
temp.loc[(temp.loc[:, 'cls'] > 30), 'gt30'] = 1
temp.loc[(temp.loc[:, 'cls'] < 30), 'gt30'] = 0
temp.loc[:, ['retdate', 'cls', 'gt30']]

,retdate,cls,gt30
2,2010-04-07,34.3714,1.0
1413,2010-04-13,34.6329,1.0
2826,2010-06-11,36.2157,1.0
4260,2010-11-23,44.1043,1.0
5738,2010-02-01,27.8186,0.0
...,...,...,...
355044,2010-11-12,44.0043,1.0
356518,2010-02-24,28.6657,0.0
357917,2010-08-13,35.5857,1.0
359368,2010-06-08,35.6186,1.0


# Main Problems  

1. Return the list of symbols that exist in 2011, but not 2010.

In [37]:
symb_2011 = df2011.loc[:, 'symb'].unique()
df2010.loc[~(df2010.loc[:, 'symb'].isin(symb_2011)), ['symb']]

,symb
73,AMPH
118,ATAC
219,CASA
245,CELH
292,COGT
...,...
815188,JPM-F
815189,JPM-G
815629,PSA-B
815635,PSA-V


2. Using the fnd data, return company name, year and the a column called “HFlag” which is equal to 1 if the company has a net income larger than $1 Billion dollars and 0 otherwise. Only include those companies whose name begins with “B”.

In [46]:
temp = dffnd.copy().loc[(dffnd.loc[:, 'conm'].str.upper().str.startswith('B'))]
temp.loc[(temp.loc[:, 'netinc'] > 1000), 'HFlag'] = 1
temp.loc[(temp.loc[:, 'netinc'] < 1000), 'HFlag'] = 0
temp.loc[:, ['conm', 'fyear', 'HFlag']]

,conm,fyear,HFlag
148,BEAM INC,2009.0,0.0
149,BEAM INC,2010.0,0.0
150,BEAM INC,2011.0,0.0
365,BRITISH AMER TOBACCO PLC,2009.0,1.0
366,BRITISH AMER TOBACCO PLC,2010.0,1.0
...,...,...,...
33719,BANCO SANTANDER BRASIL -ADR,2011.0,1.0
33752,BABYLON LTD,2010.0,0.0
33753,BABYLON LTD,2011.0,0.0
33759,BRAINSWAY LTD,2009.0,0.0


3. Using the fnd data, which ticker symbols have a net income to employee ratio greater than $1,000 in fiscal year 2010 and also have a net income between 20 and 30 million dollars in 2011?

In [51]:
(dffnd.loc[((dffnd.loc[:, 'netinc']*1000000)/(dffnd.loc[:, 'emp']*1000) > 1000) &
(dffnd.loc[:, 'fyear'] == 2010) &
(dffnd.loc[:, 'netinc'].between(20, 30)), ['tic']])

,tic
87,ATRI
185,HAIPF
265,APAGF
281,ATU
287,AROBF
...,...
33400,BLKB
33445,UCTT
33547,LXFR
33697,EXXIQ


4. Using the fnd data, which companies (company name), in fiscal year 2010 had a profit margin greater than 20%, turnover more than 2 and more than 10,000 employees?

In [55]:
(dffnd.loc[(dffnd.loc[:, 'fyear'] == 2010) &
(dffnd.loc[:, 'emp'] > 10) &
((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']) > 0.2) &
((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) > 2), ['conm']])

,conm
224,AMGEN INC
274,APPLE INC
534,RESOLUTE FOREST PRODUCTS INC
670,CANADIAN NATIONAL RAILWAY CO
855,COCA-COLA CO
992,MOLSON COORS BREWING CO
1001,CORNING INC
1874,HYDRO QUEBEC
1946,INTEL CORP
2050,SMURFIT-STONE CONTAINER CORP


5. The lowest five symbols by volume from Janaury 11th, 2010 that have a volume between 1 million and 10 million on December 1st, 2011. In other words, of those stocks which had between 1 and 10 million shares traded on December 1st, 2011, which five have the lowest volume traded on January 11th, 2010.

In [12]:
symb_2011 = df2011.loc[(df2011.loc[:, 'vol'].between(1000000, 10000000)) & (df2011.loc[:, 'retdate'] == '2011-12-01'), 'symb']
df2010.loc[(df2010.loc[:, 'symb'].isin(symb_2011)) & (df2010.loc[:, 'retdate'] == '2010-01-11'), :].nsmallest(5, 'vol')

,symb,retdate,opn,high,low,cls,vol,exch
167135,KTOS,2010-01-11,10.48,10.48,10.24,10.3100,42300,NASDAQ
166468,AMRN,2010-01-11,1.11,1.35,1.11,1.3400,50900,NASDAQ
468612,BRK.B,2010-01-11,66.64,66.87,66.16,66.4998,52400,NYSE
470035,VIV,2010-01-11,24.82,25.14,24.34,24.6100,81000,NYSE
469609,OZM,2010-01-11,15.01,15.01,14.78,14.9400,90400,NYSE


6. Of the stocks (symbols) that existed in 2011, but not in 2010, which had the highest closing price in 2011?

In [14]:
df2011.loc[~(df2011.loc[:, 'symb'].isin(df2010.loc[:, 'symb'])), :].nlargest(1, 'cls')

,symb,retdate,opn,high,low,cls,vol,exch
69593,GEVO,2011-04-07,7428.0,7908.0,7278.0,7665.0,140000,NASDAQ


7. Which symbols were in the top 500 of dollar volume on the 2nd, 3rd and 4th days of February 2011 (The stock needs to be in the top 500 for all days)?

In [18]:
top_2nd = df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-02'), :].nlargest(500, 'vol').loc[:, 'symb']
top_3rd = df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-03'), :].nlargest(500, 'vol').loc[:, 'symb']
top_4th = df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-04'), :].nlargest(500, 'vol').loc[:, 'symb']
pd.Series(df2011.loc[(df2011.loc[:, 'symb'].isin(top_2nd)) & 
           (df2011.loc[:, 'symb'].isin(top_3rd)) &
           (df2011.loc[:, 'symb'].isin(top_4th)), 'symb'].unique())

0      AAPL
1      ACLS
2      ADBE
3       ADI
4      ADSK
       ... 
392     XOM
393     XRX
394     YUM
395      HK
396     NSM
Length: 397, dtype: object

8. Of the symbols that had volume between 100,000 and 1,000,000 on the 2nd and 3rd of February 2011, which had volume greater than 5,000,000 on the 4th on February?

In [19]:
symb_2nd = df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-02') &
                       (df2011.loc[:, 'vol'].between(100000, 1000000)), 'symb']
symb_3rd = df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-03') &
                       (df2011.loc[:, 'vol'].between(100000, 1000000)), 'symb']
df2011.loc[(df2011.loc[:, 'retdate'] == '2011-02-04') &
           (df2011.loc[:, 'vol'] > 5000000), :]

,symb,retdate,opn,high,low,cls,vol,exch
3067,AAPL,2011-02-04,49.0871,49.5286,49.0729,49.50,11494200,NASDAQ
3080,ACLS,2011-02-04,10.5200,11.2800,9.9200,11.12,8165400,NASDAQ
3088,ADBE,2011-02-04,33.5400,33.7200,33.1700,33.36,6536700,NASDAQ
3128,ALTR,2011-02-04,39.5700,41.1100,39.4800,41.01,7673300,NASDAQ
3131,AMAT,2011-02-04,16.1500,16.5000,16.0200,16.49,15023900,NASDAQ
...,...,...,...,...,...,...,...,...
534450,WU,2011-02-04,20.6900,21.0000,20.6800,20.96,5072300,NYSE
534453,WY,2011-02-04,23.9900,24.7500,23.9400,24.30,10446900,NYSE
534455,X,2011-02-04,61.1200,61.4000,57.9600,58.17,14977600,NYSE
534460,XOM,2011-02-04,83.2500,83.5200,82.8500,83.28,16559900,NYSE


9. Generate the following dataset:  
• company name, ticker symbol, revenue for all companies whose name begin with “A” or “a”   
• A column, revflag which is 1 if revenue is greater than $25,000,000 and 0 otherwise.  

In [21]:
temp = dffnd.copy().loc[(dffnd.loc[:, 'conm'].str.upper().str.startswith('A')), 
                        ['conm', 'tic', 'rev']]
temp.loc[:, ['revflag']] = 0
temp.loc[(temp.loc[:, 'rev'] > 25), ['revflag']] = 1
temp

/var/folders/dy/xs5p_qy53c58lk4l5yndb1n00000gr/T/ipykernel_24275/1331084049.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  temp.loc[:, ['revflag']] = 0


,conm,tic,rev,revflag
0,AAR CORP,AIR,1423.976,1
1,AAR CORP,AIR,1352.151,1
2,AAR CORP,AIR,1775.782,1
3,ADC TELECOMMUNICATIONS INC,ADCT,996.700,1
4,ADC TELECOMMUNICATIONS INC,ADCT,1156.600,1
...,...,...,...,...
33780,AVG TECHNOLOGIES NV,AVG,272.392,1
33799,AVIANCA HOLDINGS SA,AVH,3794.428,1
33813,ALLEGION PLC,ALLE,NaN,0
33814,ALLEGION PLC,ALLE,NaN,0


10. Generate the following dataset:  
• company name, ticker symbol, revenue, inventory and employee information from fiscal year 2010  
• A column called turnflag which is 1 for companies with turnover greater than 2, 0 otherwise  
• For a company to be included it must have revenue, inventory and employee all greater than
zero for both 2010 and 2011

In [25]:
conm_2011 = dffnd.loc[((dffnd.loc[:, 'fyear'] == 2011) & 
                         (dffnd.loc[:, 'rev'] > 0) &
                         (dffnd.loc[:, 'emp'] > 0) &
                         (dffnd.loc[:, 'invt'] > 0)),
                         'conm'
                        ]
temp = dffnd.copy().loc[(dffnd.loc[:, 'fyear'] == 2010) & (dffnd.loc[:, 'conm'].isin(conm_2011)), 
                        ['conm', 'tic', 'rev', 'invt', 'emp']]
temp.loc[:, ['turnflag']] = 0
temp.loc[(~(temp.loc[:, 'rev'].isna())) &
          (~(temp.loc[:, 'invt'].isna())) &
          (temp.loc[:, 'invt'] > 0) &
          (temp.loc[:, 'emp'] > 0) &
          (temp.loc[:, 'rev'] > 0) &
          ((temp.loc[:, 'rev']/temp.loc[:, 'invt']) > 2), ['turnflag']] = 1
temp

/var/folders/dy/xs5p_qy53c58lk4l5yndb1n00000gr/T/ipykernel_24275/3000923034.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  temp.loc[:, ['turnflag']] = 0


,conm,tic,rev,invt,emp,turnflag
6,AMERICAN AIRLINES GROUP INC,AAL,22170.000,594.000,78.250,1
9,CECO ENVIRONMENTAL CORP,CECE,140.602,12.777,0.556,1
18,PINNACLE WEST CAPITAL CORP,PNW,3263.645,202.989,6.740,1
21,AARON'S INC,AAN,1876.847,11.200,10.400,1
24,ABBOTT LABORATORIES,ABT,35166.721,3188.734,90.000,1
...,...,...,...,...,...,...
33750,ECOPETROL SA,EC,21517.153,1123.884,6.744,1
33772,LYONDELLBASELL INDUSTRIES NV,LYB,41151.000,4824.000,14.000,1
33776,OCEAN RIG UDW INC,ORIG,405.712,0.000,0.445,0
33779,AVG TECHNOLOGIES NV,AVG,217.218,0.523,0.630,1


# More Problems

5. Of the stocks (ticker symbols) that have a net-income to employee ratio greater than $1,000 in fiscal year 2010 and more than 1,000 employees in 2011, what is the lowest profit margin in fiscal year 2011?

In [75]:
tics_2010 = dffnd.loc[((dffnd.loc[:, 'netinc']*1000000)/(dffnd.loc[:, 'emp']*1000) > 1000) & (dffnd.loc[:, 'fyear'] == 2010), 'tic']
(dffnd.loc[
    (dffnd.loc[:, 'tic'].isin(tics_2010)) &
    (dffnd.loc[:, 'emp'] > 1) &
    (dffnd.loc[:, 'fyear'] == 2011) &
    (dffnd.loc[:, 'rev'] != 0)
, :].
assign(prof_marg = dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']).
nsmallest(1, 'prof_marg').
loc[:, ['prof_marg']])

,prof_marg
20947,-2.131629


6. Of the stocks (ticker symbols) that have a net-income to employee ratio greater than $1,000 in fiscal year 2010 and between 1,000 and 2,000 employees in 2011, what is the highest profit margin in fiscal year 2011 and what is the ticker symbol?

In [76]:
tics_2010 = dffnd.loc[((dffnd.loc[:, 'netinc']*1000000)/(dffnd.loc[:, 'emp']*1000) > 1000) & (dffnd.loc[:, 'fyear'] == 2010), 'tic']
(dffnd.loc[
    (dffnd.loc[:, 'tic'].isin(tics_2010)) &
    (dffnd.loc[:, 'emp'].between(1, 2)) &
    (dffnd.loc[:, 'fyear'] == 2011) &
    (dffnd.loc[:, 'rev'] != 0)
, :].
assign(prof_marg = dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']).
nlargest(1, 'prof_marg').
loc[:, ['tic', 'prof_marg']])

,tic,prof_marg
11005,BOWFF,2.898055



7. Of the companies (ticker symbols) with turnover between 1 and 2 in 2010, which companies also had a net income to employee ratio greater than $1,000 in 2010?

In [77]:
(dffnd.loc[
    (dffnd.loc[:, 'fyear'] == 2010) &
    ((dffnd.loc[:, 'netinc']*1000000)/(dffnd.loc[:, 'emp']*1000) > 1000) &
    (dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt'].between(1, 2))
, ['tic']])

,tic
2,AIR
4,ADCT
9,CECE
16,AVX
18,PNW
...,...
33769,INXN
33772,LYB
33776,ORIG
33779,AVG


8. Of the companies (ticker symbols) with turnover between 1 and 2 in 2010, which companies also had a net income to employee ratio greater than $1,000 in 2011?

In [79]:
tic_2011 = dffnd.loc[
    (dffnd.loc[:, 'fyear'] == 2011) &
    ((dffnd.loc[:, 'netinc']*1000000)/(dffnd.loc[:, 'emp']*1000) > 1000), 'tic']
(dffnd.loc[
    (dffnd.loc[:, 'tic'].isin(tic_2011)) &
    (dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt'].between(1, 2)) &
    (dffnd.loc[:, 'fyear'] == 2010)
, ['tic']])

,tic
9,CECE
18,PNW
21,AAN
24,ABT
33,ACET
...,...
33762,ORXGF
33769,INXN
33772,LYB
33776,ORIG


9. Write a select statement to generate the following datasets:  
• company name, ticker symbol, revenue, inventory and employee information from both 2010 and 2011 fiscal years.  
• A column called invtflag which is equal to 1 for companies with turnover between 2 and 3, 2 for turnover between 3 and 4 and 5 for turnover greater than 4 and zero otherwise.  
• A column called invtProfit which is equal to 1 for companies with less than 20% profit margin and turnover greater than 2, 2 for companies with profit margin greater than 40% and turnover greater than 2 and 0 otherwise.  
• A column called EmployeeProfit which is equal to 0 for companies that have profit margins between 20% and 40% and have more than 10,000 employees, is equal to a company’s profit margin if the margin is less than 20%, is equal to twice the number of employees (if it exists) if the profit margin is greater than 40% and is -1 otherwise.  

In [104]:
temp = dffnd.copy()
# invtflag
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']).between(2, 3)), 'invtflag'] = 1
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']).between(3, 4)), 'invtflag'] = 2
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) > 4), 'invtflag'] = 5
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) < 2), 'invtflag'] = 0
# invtProfit
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) > 2) & ((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']) < 0.2), 'invtProfit'] = 1
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) > 2) & ((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']) > 0.4), 'invtProfit'] = 2
temp.loc[((dffnd.loc[:, 'rev']/dffnd.loc[:, 'invt']) < 2) & ((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']).between(0.2, 0.4)), 'invtProfit'] = 0
# EmployeeProfit
temp.loc[(dffnd.loc[:, 'emp'] > 10) & ((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']).between(0.2, 0.4)), 'EmployeeProfit'] = 0
temp.loc[((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']) < 0.20), 'EmployeeProfit'] = dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']
temp.loc[((dffnd.loc[:, 'netinc']/dffnd.loc[:, 'rev']) > 0.40) & (~(dffnd.loc[:, 'emp'].isna())), 'EmployeeProfit'] = dffnd.loc[:, 'emp']*2
temp.loc[((dffnd.loc[:, 'emp'].isna()) & (dffnd.loc[:, 'netinc'].isna()) & (dffnd.loc[:, 'rev'].isna())), 'EmployeeProfit'] = -1

temp.loc[:, ['conm', 'tic', 'rev', 'invt', 'emp', 'invtflag', 'invtProfit', 'EmployeeProfit']]

,conm,tic,rev,invt,emp,invtflag,invtProfit,EmployeeProfit
0,AAR CORP,AIR,1423.976,477.424,5.30,1.0,1.0,0.055233
1,AAR CORP,AIR,1352.151,496.904,5.80,1.0,1.0,0.033005
2,AAR CORP,AIR,1775.782,507.274,6.10,2.0,1.0,0.039321
3,ADC TELECOMMUNICATIONS INC,ADCT,996.700,131.100,9.05,5.0,1.0,-0.475870
4,ADC TELECOMMUNICATIONS INC,ADCT,1156.600,106.400,9.30,5.0,1.0,0.053605
...,...,...,...,...,...,...,...,...
33812,CARACAL ENERGY INC,CCAXF,NaN,NaN,NaN,NaN,NaN,-1.000000
33813,ALLEGION PLC,ALLE,NaN,NaN,NaN,NaN,NaN,-1.000000
33814,ALLEGION PLC,ALLE,NaN,NaN,NaN,NaN,NaN,-1.000000
33815,ALLEGION PLC,ALLE,2021.200,254.500,NaN,5.0,1.0,0.107906


10. What are the symbols and dollar volume traded for the companies with the top 5 dollar volume traded (based on closing price) on February 3rd 2010 (NYSE only)?

In [105]:
df2010.loc[
    (df2010.loc[:, 'retdate'] == '2010-02-03') &
    (df2010.loc[:, 'exch'] == 'NYSE')
    ].assign(dol_vol = df2010.loc[:, 'cls']*df2010.loc[:, 'vol']).nlargest(5, 'dol_vol').loc[:, ['symb', 'dol_vol']]

,symb,dol_vol
521811,PNC,4.011385e+09
521586,MSI,3.537079e+09
521223,GS,2.639860e+09
521770,PFE,2.122732e+09
520644,BAC,2.032279e+09
